<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [5]:
import pandas as pd

pd.options.display.max_columns = 50

In [6]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
for column in df.columns:
    print(df[column].value_counts(dropna=False))
    print("\n")
    
# bunch of binaries/categories that need encoding
# plus something hinky going on with TotalCharges

1565-RHDJD    1
3797-FKOGQ    1
2521-NPUZR    1
7590-VHVEG    1
8809-RIHDD    1
4712-UYOOI    1
2710-WYVXG    1
2458-EOMRE    1
4102-OQUPX    1
9961-JBNMK    1
0707-HOVVN    1
3933-DQPWX    1
5167-ZFFMM    1
7876-BEUTG    1
6347-DCUIK    1
1262-OPMFY    1
3006-XIMLN    1
3280-NMUVX    1
1837-YQUCE    1
6733-LRIZX    1
2055-BFOCC    1
5353-WILCI    1
3744-ZBHON    1
3746-EUBYR    1
0071-NDAFP    1
7872-RDDLZ    1
4566-NECEV    1
7209-JCUDS    1
7598-UAASY    1
3824-RHKVR    1
             ..
9758-MFWGD    1
9453-PATOS    1
7153-CHRBV    1
3393-FMZPV    1
5680-LQOGP    1
7611-YKYTC    1
5928-QLDHB    1
0022-TCJCI    1
7636-OWBPG    1
5550-VFRLC    1
5609-IMCGG    1
0404-SWRVG    1
5655-JSMZM    1
7140-ADSMJ    1
0961-ZWLVI    1
5515-RUGKN    1
6563-VRERX    1
8510-AWCXC    1
7771-ZONAT    1
6994-ORCWG    1
8148-BPLZQ    1
1229-RCALF    1
7781-EWARA    1
6235-VDHOM    1
5423-BHIXO    1
8128-YVJRG    1
2697-NQBPF    1
6331-EWIEB    1
0224-HJAPT    1
7549-MYGPK    1
Name: customerID, Length

In [7]:
import numpy as np

df['TotalCharges'] = df['TotalCharges'].replace(" ", np.nan)

df['TotalCharges'].describe()

count     7032
unique    6530
top       20.2
freq        11
Name: TotalCharges, dtype: object

In [8]:
df['TotalCharges'] = df['TotalCharges'].apply(float)

In [8]:
df['TotalCharges'].describe()

count    7032.000000
mean     2283.300441
std      2266.771362
min        18.800000
25%       401.450000
50%      1397.475000
75%      3794.737500
max      8684.800000
Name: TotalCharges, dtype: float64

In [9]:
df = df.drop(columns=["customerID"])

df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
!pip install category_encoders

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
from category_encoders.ordinal import OrdinalEncoder

oe = OrdinalEncoder()

data = oe.fit_transform(df)

data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,1
1,2,0,2,1,34,2,2,1,2,2,2,1,1,1,2,2,2,56.95,1889.50,1
2,2,0,2,1,2,2,2,1,2,1,1,1,1,1,1,1,2,53.85,108.15,2
3,2,0,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75,1
4,1,0,2,1,2,2,2,2,1,2,1,1,1,1,1,1,1,70.70,151.65,2


In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)

train.shape, test.shape

((5282, 20), (1761, 20))

In [13]:
X_train = train.drop(columns=['Churn'])
X_test = test.drop(columns=['Churn'])

y_train = train['Churn']
y_test = test['Churn']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5282, 19), (5282,), (1761, 19), (1761,))

In [14]:
# make it 0/1
y_train = y_train - 1
y_test = y_test - 1

In [3]:
import tensorflow.keras
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [15]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train.values, y_train.values)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 
# batch size 10

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Best: 0.7370314320259499 using {'batch_size': 10, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7370314320259499, Stdev: 0.01355144969898908 with: {'batch_size': 100, 'epochs': 20}


In [65]:
param_grid = {'batch_size': [10],
              'epochs': [20],
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train.values, y_train.values)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7322983812965348 using {'batch_size': 10, 'epochs': 20}
Means: 0.7322983812965348, Stdev: 0.006397458358494454 with: {'batch_size': 10, 'epochs': 20}


In [14]:
!pip install wandb

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
!wandb login a032c0bf565c8f0b245e5f5c1b76c761bbf2a003

wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
Successfully logged in to Weights & Biases!


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb: WARNING import wandb.keras called before import keras or import tensorflow.keras.  This can lead to a version mismatch, W&B now assumes tensorflow.keras


In [4]:
wandb.init(project="u4-m4-assignment", entity="srfinley")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


W&B Run: https://app.wandb.ai/srfinley/u4-m4-assignment/runs/qjkeed4h

In [17]:
wandb.config.epochs = 50
wandb.config.batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(12, activation='relu', input_dim=19))
model.add(Dense(1, activation='sigmoid'))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X_train.values, y_train.values, 
          validation_split=0.1, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
Error generating diff: Command '['git', 'diff', '--submodule=diff', 'HEAD']' timed out after 5 seconds


Train on 4753 samples, validate on 529 samples
Epoch 1/50
4753/4753 [==============================] - 2s 373us/sample - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 2/50
4753/4753 [==============================] - 1s 219us/sample - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 3/50
4753/4753 [==============================] - 1s 213us/sample - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 4/50
4753/4753 [==============================] - 1s 216us/sample - loss: nan - mean_squared_error: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_mean_absolute_error: nan
Epoch 5/50
4753/4753 [==============================] - 1s 212us/sample - loss: nan - mean_square

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?